In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import matplotlib.pyplot as plt
from functools import partial
import numpy as np

In [ ]:
import sys

sys.path.append("..")

In [ ]:
# Import the various experiments we need for the notebook
from sva.experiments import GPDream

# Plotting quality of life utils
from sva.mpl_utils import set_mpl_defaults

# Import the campaign
from sva.campaign import Campaign, CampaignData, FixedPolicy

# Model to use
from sva.models import EasySingleTaskGP

In [ ]:
set_mpl_defaults()

# A simple 2d example campaign

In [ ]:
SEED = 131
N_start = 5
N_max = 40
ppd = 200

In [ ]:
experiment = GPDream.from_default(gp_model_params={"kernel":"rbf", "lengthscale": 0.1}, d=2, seed=SEED)
x = experiment.get_dense_coordinates(ppd=ppd)
y = experiment(x)
extent = experiment.get_domain_mpl_extent()

We'll need some other bits and pieces to effectively run a campaign!

In [ ]:
data = CampaignData()
data.prime(experiment, "LatinHypercube", seed=SEED, n=N_start)
policy = FixedPolicy(n_max=N_max, acquisition_function="EI", save_model=True)
campaign = Campaign(data=data, experiment=experiment, policy=policy, seed=SEED)

In [ ]:
campaign.run()

In [ ]:
model = campaign.get_model_at_iteration(-1)
pred, std = model.predict(x)
experiment_maxima = experiment.metadata["optimum"][1].item()
experiment_argmax = experiment.metadata["optimum"][0].numpy()

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(6, 2), sharex=True, sharey=True)
plot_kwargs = {"extent": extent, "interpolation": "nearest", "origin": "lower", "cmap": "viridis"}

ax = axs[0]
ax.imshow(y.reshape(ppd, ppd).T, **plot_kwargs)
ax.scatter(data.X[:N_start, 0], data.X[:N_start, 1], color='black')
ax.scatter(data.X[:, 0], data.X[:, 1], s=3, color='red')
ax.scatter(experiment_argmax[:, 0], experiment_argmax[:, 1], marker="x", s=10)
ax.set_title("Truth w/ samples")

ax = axs[1]
ax.imshow(pred.reshape(ppd, ppd).T, **plot_kwargs)
ax.set_title("Pred")

ax = axs[2]
plot_kwargs["cmap"] = "RdBu"
ax.imshow((y - pred.reshape(-1, 1)).reshape(ppd, ppd).T, **plot_kwargs)
ax.set_title("Delta")

ax = axs[3]
plot_kwargs["cmap"] = "viridis"
ax.imshow(std.reshape(ppd, ppd).T, **plot_kwargs)
ax.set_title("Std")

plt.show()

We can also plot a few key metrics to get an idea of how well the models are performing.

In [ ]:
# Relative opportunity cost
model_maxima_x = np.array([md["model_optimum"][0].numpy() for md in campaign.data.metadata[N_start:]]).squeeze()
experiment_at_model_maxima_x = experiment(model_maxima_x)
relative_opportunity_cost = np.abs(experiment_maxima - experiment_at_model_maxima_x) / np.abs(experiment_maxima)

# Values of the points themselves
sampled_y_values = campaign.data.Y.squeeze()
relative_sampled_y_values_cost = np.abs(experiment_maxima - sampled_y_values) / np.abs(experiment_maxima)
relative_sampled_y_values_cost = [np.min(relative_sampled_y_values_cost[:ii]) for ii in range(N_start, N_max)]

# Other stuff...

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(3, 2), sharex=True, sharey=True)

x = np.arange(N_start, N_max) + 1


ax = axs[0]
ax.plot(x, relative_opportunity_cost)
ax.set_ylabel("Rel. O.C.")

ax = axs[1]
ax.plot(x, relative_sampled_y_values_cost)
ax.set_xlabel("$N$")

for ax in axs:
    ax.set_yscale("log")

plt.show()